# Do we have a business case?

## Bronnen

**Geschiedenis PostNL**

- https://www.postnl.nl/over-postnl/over-ons/geschiedenis


**Brievenbuspost**

- https://www.acm.nl/nl/publicaties/acm-omzet-uit-brievenbuspost-2021-voor-het-eerst-jaren-gestegen

**MNIST Digit Classifier using PyTorch**

- https://tomytjandra.github.io/blogs/python/classification/computer-vision/pytorch/2021/08/27/mnist-digit-pytorch.html

**3blue1brown: But what is a neural network?**

- https://www.youtube.com/watch?v=aircAruvnKk

## Context

We schrijven het jaar 1990, het (fictieve) jaar waarin PTT Post aankondigt om over te stappen op het automatisch scannen van postcodes. De startup FastScan heeft een prototype waarmee ze postcodes kunnen scannen op grote snelheid.

Namens de startup FastScan krijgen we de opdracht om te onderzoeken of dit een business case voor hen is. Ze verwachten van ons een gedegen onderzoek en vragen ons om de case samen te vatten.

## Opdrachten

### Opdracht 1: Voorbereiding

FastScan heeft een Notebook voor ons klaargezet om onze research mee te starten.

In [2]:
# a. Open het Notebook 2021-08-27-mnist-digit-pytorch.ipynb

# b. Check het kopje 'Installation' of je alle onderdelen geinstalleerd hebt

# c. Draai de cel 'Libraries' en installeer eventueel ontbrekende onderdelen

# d. Bekijk het kopje 'Workflow' om een beeld te krijgen van het proces

### Opdracht 2: Data inlezen en visualiseren

FastScan heeft het ons makkelijk gemaakt. De afbeeldingen van handgeschreven getallen staan in een aantal csv-bestanden die we kunnen inlezen en kunnen visualiseren.

- Let op: het is niet noodzakelijk om alle details te begrijpen, maar wel de grote lijn

Vraag om hulp als dit niet lukt.

In [3]:
# a. Doorloop de stappen van 'Load Data'

# b. Doorloop de stappen van 'Visualize Data'

### Opdracht 3: Trainen en valideren

FastScan heeft uitvoerig beschreven hoe we hun model kunnen trainen en kunnen valideren.

- Let op: het is niet noodzakelijk om alle details te begrijpen, maar wel de grote lijn

Vraag om hulp als dit niet lukt.

- Fun fact: het fully-connected neurale netwerk dat FastScan heeft gebouwd voldoet geheel aan visualisatie van 3blue1brown. Kijk dit eventueel terug om meer grip te krijgen om het netwerk van FastScan.

In [ ]:
# a. Doorloop de stappen van 'Define Model Architecture'

# b. Doorloop de stappen van 'Train the Model'

# c. Doorloop de stappen van 'Test the Model'

# d. Check dat het model opgeslagen is in ./cache/model-mnist-digit.pt

### Opdracht 4: Accuracy

Het models van FastScan heeft een accuracy van 97%. Dat klinkt hoog maar heeft dit business value?

Tijd voor een eerste analyse!

In [9]:
# a. Voor de zekerheid: wat is de definitie van accuracy?
#   de vergelijking van het antwoord van het model met de goede antwoorden.

# b. Lees het ACM artikel over de brievenbuspost

#    - Hoeveel brievenbuspost wordt er per jaar in totaal verstuurd?
#   rond de 2 miljard

# c. Hoeveel postcodes (en dus brieven) worden er bij een accuracy van 97% correct gelezen?

#    - Hint: hoeveel nummers bevat een postcode?
#   4

#    - Gebruik verder dat de verwachtingswaarde = kans op succes x aantal
#   0.97*0.97*0.97*0.97 = 0.88529281 -> 89%

#   2.000,000,000 * 0.89 = 1,780,000,000

# d. En hoeveel dus incorrect?
#   220,000,000

# e. PPT Post eist dat slechts 1% van de brieven foutief gelezen mag worden
#   1% = 20,000,000

#   - Hoeveel moet de accuracy van ons model dan worden?
#   100%

# f. Check eens online wat de hoogste accuracy op deze dataset is
#  99.41% is het hoogst wat ik gevonden heb op https://towardsdatascience.com/going-beyond-99-mnist-handwritten-digits-recognition-cfff96337392

#   - Voldoet dat model aan de PTT Post eis?
#   0.9941 * 0.9941 * 0.9941 * 0.9941 = 0.9766080396957361 -> dus nee.


### Opdracht 5: Compute tijd en kosten

We zijn klaar voor een tweede analyse: de compute time.

Oftewel: hoeveel tijd kost het om met het model voorspellingen te doen.

In [5]:
# a. Laad het opgeslagen model in (zoek in het Notebook hoe dit werkt)
import pandas as pd  # for read_csv
import numpy as np  # for np.Inf
import matplotlib.pyplot as plt  # visualization
import seaborn as sns  # heatmap visualization
from sklearn.metrics import confusion_matrix, classification_report  # evaluation metrics
import pickle  # serialization

import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

with open('./cache/history-mnist-digit.pickle', 'rb') as f:
    history = pickle.load(f)

# load the best model
model.load_state_dict(torch.load('./cache/model-mnist-digit.pt'))

#    - Zet het model in evaluate mode met de eval() functie
y_test = []
y_pred = []

# disable gradient calculation
with torch.no_grad():
    # prepare model for evaluation
    model.eval()

    # loop for each data
    for data, target in test_loader:
        # STEP 1: forward pass
        output = model(data)
        # STEP 2: get predicted label
        _, label = torch.max(output, dim=1)
        # STEP 3: append actual and predicted label
        y_test += target.numpy().tolist()
        y_pred += label.numpy().tolist()
#      Hier niet noodzakelijk maar wel in de meeste gevallen:

#     "Remember that you must call model.eval() to set dropout and
#      batch normalization layers to evaluation mode before running inference.
#      Failing to do this will yield inconsistent inference results."

#      Bron: https://pytorch.org/tutorials/beginner/saving_loading_models.html#save-load-entire-model

# b. Laad de test dataset 'test.csv' in (zoek in het Notebook hoe dit werkt)
class MNISTDataset(Dataset):
    # constructor
    def __init__(self, file_path):
        # read data
        df = pd.read_csv(file_path)

        target_col = "label"
        self.label_exist = target_col in df.columns

        if self.label_exist:
            # split feature-target
            X = df.drop(columns=target_col).values
            y = df[target_col].values

            # convert numpy array to tensor
            self.y = torch.tensor(y)
        else:
            X = df.values

        self.X = torch.tensor(X, dtype=torch.float32)

        # scaling
        self.X /= 255

    # for iteration
    def __getitem__(self, idx):
        if self.label_exist:
            return self.X[idx], self.y[idx]
        else:
            return self.X[idx]

    # to check num of observations
    def __len__(self):
        return self.X.size()[0]
# load test.csv
mnist_data_unlabeled = MNISTDataset("./mnist/test.csv")
unlabeled_loader = DataLoader(mnist_data_unlabeled, batch_size=1, shuffle=True)

print(mnist_data_unlabeled.X.size())

#    - Zet de batch size op 1 zodat we de afbeeldingen één voor één kunnen doorlopen

# c. Doorloop de test set eenmaal en meet hoeveel tijd 1000 inferences gemiddeld kosten

import time
# loop data
start = time.time()
for idx in range(1):
    start2 = time.time()
    # get image from loader
    image = next(iter(unlabeled_loader))
    image = image.reshape((28, 28))

    ##############
    # prediction #
    ##############

    with torch.no_grad():
        # forward pass
        output = model(image.reshape(1, -1))
        # calculate probability
        prob = F.softmax(output, dim=1)

    #################
    # visualization #
    #################

    fig, axes = plt.subplots(1, 2, figsize=(8, 4))

    # show digit image
    axes[0].imshow(image, cmap="gray")
    axes[0].axis("off")

    # predicted probability barplot
    axes[1].barh(range(0, 10), prob.reshape(-1))
    axes[1].invert_yaxis()
    axes[1].set_yticks(range(0, 10))
    axes[1].set_xlabel("Predicted Probability")

    plt.tight_layout()
    plt.show()

    end2 = time.time()
    print(end2 - start2)

end = time.time()
print(end - start)

#    - Met welke module kun je tijdmetingen doen?
#   met de module time

#    - Zoek in het Noteboek hoe je een forward pass uitvoert met het model

# d. Lees nogmaals het ACM artikel over de brievenbuspost

#    - Hoeveel brievenbuspost wordt er per jaar in totaal verstuurd?

#    - Hoeveel is dat per dag?

#    - Hoeveel nummer scans zijn dat?

#    - Hoeveel tijd kost het ons om dat met 1 CPU uit te voeren?

# e. Stel PTT Post verwacht dat we de scans binnen 1 uur moeten uitvoeren:

#    - Gaat dit lukken?

# f. Dit is een zeer versimpelde voorstelling van zaken.

#    - Welke praktische factoren zouden we ook mee moeten nemen?

#    - Hint: maak een voorstelling hoe we de scans op brieven en pakketen zouden uitvoeren

# g. Stel dat we een factor 100_000 te laag zitten

#   - Hoeveel tijd hebben we dan nodig voor een dag brievenbuspost?

# h. Stel dat we het proces met een factor 100 moeten versnellen:

#   - Hoe zouden we dat kunnen doen zonder een GPU te gebruiken?

AttributeError: 'dict' object has no attribute 'load_state_dict'

Om onze compute te hosten hebben we meerdere opties:

- Self hosting (wij beheren de servers)
- Cloud functions (geen beheer)

Hieronder een aantal self hosting opties:

- Vultr: https://www.vultr.com/pricing/#optimized-cloud-compute
- Paperspace: https://www.paperspace.com/pricing

In [12]:
# i. Kijk eens wat de CPU compute kosten zijn bij Vultr en Paperspace

# j. Zijn ze te makkelijk te vergelijken?

# k. Hoe zouden we een eerlijke vergelijking kunnen maken?

#    - Hit: zie opdracht 5c

# l. Zouden we op die manier ook de kosten kunnen ramen?

Google Compute functions zijn een voorbeeld van het aanroepen van puur een functie,

bijvoorbeeld voor de inference van ons model. Bekijk dit rekenvoorbeeld eens:

- https://cloud.google.com/functions/pricing#high_volume_http_function

In [11]:
# m. Hoeveel zouden de kosten voor ons zijn als we dit rekenvoorbeeld volgen?

# n. De kosten zijn een stuk hoger dan wanneer we self hosting toepassen

#    - Zoek eens op wat een gemiddelde systeembeheerder kost per jaar

#    - Als je die kosten meeneemt, moeten we dan self hosting of cloud functions gebruiken?

## Conclusie

Er komt veel kijken bij het analyseren van een business case voor postcode scans. We zouden nog mee moeten nemen:

- Wat de personeelskosten van FastScan totaal zijn
- Wat de kostenbesparing voor PTT Post is bij verschillende accuracies
- ...

Op basis van bovenstaande analyse hebben we wel meer inzicht verkregen, maar we zijn verre van compleet.

Kunnen we wel iets van advies geven?